In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
from scipy.special import comb, perm

extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [3]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
    Amplit = gavgfix*typenum
    numsample = typenum**2
    Amplitg= np.zeros(numsample)
    idxc=0
    while (1):
      if idxc>=numsample:
        Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
        break
      p=np.random.random(1)
      Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
      if np.sum(Amplitg)>1.0:
        continue 
      elif np.sum(Amplitg)==1.0:
        break
      else:
        idxc +=1
        # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
    # Amplitg=0
    Amplitg*=Amplit
    Amplitg=np.sqrt(Amplitg)
    return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

import math
import cmath
def ThreeSquare(x):
    if x.imag == 0:
        m = x.real
        ans = -math.pow(-m,1/3) if m<0 else math.pow(m,1/3)
    else:
        ans = x**(1/3)
    return ans

def RoundAns(x,num):
    if x.imag == 0:
        m = x.real
        ans = round(m,num)
    else:
        m = round(x.real,num)
        n = round(x.imag,num)
        ans = complex(m,n)
    return ans
    
def Cubic(args):
    a,b,c,d = args
    p = c/a-b**2/(3*a**2)
    q = d/a+2*b**3/(27*a**3)-b*c/(3*a**2)
    w = complex(-0.5,(3**0.5)/2)
    ww = complex(-0.5,-(3**0.5)/2)
    A = cmath.sqrt((q/2)**2+(p/3)**3)
    B = ThreeSquare(-q/2+A)
    C = ThreeSquare(-q/2-A)
    y1 = B+C
    y2 = w*B+ww*C
    y3 = ww*B+w*C
    D = b/(3*a)
    roots=[RoundAns(y1-D,6),RoundAns(y2-D,6),RoundAns(y3-D,6)]
    return roots


In [317]:
'''
WITH SYMMETRY
'''
'''
Cases 3,4 with symmetry
Focus on mean 
Focus on second moment
Focus on derivative on 
'''
# generate mean matrix
Nt=np.array([750,750])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
'''
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
'''
nrank,ntrial,neta,nrec=2,20,10,2

## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' 
---------- method 2 ------------------ 
'''
''' ## Three \bar{J} cases '''
JI,JE,a,b=0.575,1.225,0,0
a,b=0.025,-0.075
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
# ''' method 1 '''
# eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
''' method 2 '''
eigvAm,eigvecrAm=scpla.eig(Am,left=False,right=True)
eigvAm_,eigvecltAm = la.eig(Am.copy().T)
eigveclAm = eigvecltAm.T

meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    print('normitems',normalizecoe,'; eigenvalue:',eigvAm[i],eigvAm_[i])
    neigvecAm[:,i]=neigvecAm[:,i]*(eigvAm[i]/normalizecoe)
for i in range(nrank,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]=neigvecAm[:,i]/normalizecoe
''' for reference '''
xAm,yAm = np.zeros((NE*2,nrank),dtype=complex),np.zeros((NE*2,nrank),dtype=complex)
# for irank in range(nrank):
#     xAm[:,irank],yAm[:,irank]=meigvecAm[:,irank].copy(),neigvecAm[:,irank].copy()/eigvAm[irank]
# print('eigenvalues of unperturbed matrix:',eigvAm[:nrank])

for irank in range(nrank):
    xAm[:NE,irank],yAm[:NE,irank]=1/np.sqrt(NE*2),2*JE/(JE-JI)/np.sqrt(NE*2)
    xAm[NE:,irank],yAm[NE:,irank]=1/np.sqrt(NE*2),-2*JI/(JE-JI)/np.sqrt(NE*2)
print('eigenvalues of unperturbed matrix:',eigvAm[:nrank])

# print('unpertubed eigenvectors, left:',yAm[1::NE,0],yAm[1::NE,1])
# print('unpertubed eigenvectors, right:',xAm[1::NE,0],xAm[1::NE,1])
print('test normal unperturbed:',np.sum(yAm[:,0]*xAm[:,0]),np.sum(yAm[:,1]*xAm[:,1]))

'''
Network Setting for Iterating Following
'''
gaverage = 0.35
signeta=np.ones(3)
signeta[2]*=(-1)
## heterogeneous random gain
''' >>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>> '''
xee,xei,xie,xii=1.0,1.0,1.0,1.0 ## HOMOgeneous random gain g_kl
# xee,xei,xie,xii=1.0,0.5,0.2,0.8 ## HETEROgeneous random gain
coeffeta = np.ones(3)
# coeffeta=np.array([1.0,1.0,0.0]) ## HETERogeneous symmetry D_{\rho}

'''
Recording Variables
'''
# original data
eigvalseries=np.zeros((neta,ntrial,N),dtype=complex)
Reigvecseries,Leigvecseries=np.zeros((neta,ntrial,NE*2,nrank*nrec),dtype=complex),np.zeros((neta,ntrial,NE*2,nrank*nrec),dtype=complex)
Beigvseries=np.zeros((neta,ntrial,N),dtype=complex)
''' All random matrices for each trials '''
## Not X total (symmetry), nut Y independent
gYbasexee=np.zeros((ntrial,NE*2,NE*2),dtype=complex)

''' Iterative Processing '''
etaseries = np.linspace(0.0,1.0,neta)
for iktrial in range(ntrial):
    '''
    ##>>>>>>>>>>> g0!=0, >>>>>>>>
    '''
    Xsym=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
    Xsym[:NE,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[:NE,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    XsymT=Xsym.copy().T
    X0=Xsym.copy()  # always make mistake here
    J0=Am.copy()+X0.copy()
    ''' # calculate the original \bar{m} and \bar{n} '''
    eigvJ0,eigrvecJ0=la.eig(J0)
    inveigrvecJ0=la.inv(eigrvecJ0)
    meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    neig0=neig0.copy().T
    print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
    for i in range(nrank):
        neig0[:,i]*=eigvJ0[i]
    for j in range(nrank,N):
        neig0[:,j]*=eigvJ0[j]
    leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    reig0=meig0.copy()
    normval=np.sum(reig0*neig0.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    leig0=neig0.copy()/normval.copy()

    # print('test normal 2:',np.sum(leig0[:,1]*reig0[:,1]))
    # print('test normal 3:',np.sum(meig0[:,1]*neig0[:,1]))
    # print('eigenvalues:',eigvJ0[:nrank])
    '''## for reference '''
    x0,y0=np.zeros((NE*2,nrank),dtype=complex),np.zeros((NE*2,nrank),dtype=complex)
    for irank in range(nrank):
        x0[:,irank],y0[:,irank]=reig0[:,irank].copy(),leig0[:,irank].copy()
        ''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
        tildex,tildey=x0[:,irank].copy(),y0[:,irank].copy()
        ## make sure the E in y is positive, if negative then all x and y should change sign
        if (np.mean(tildey[:NE])*yAm[0,irank])<0:
            tildex*=(-1)
            tildey*=(-1)
        x0[:,irank],y0[:,irank]=tildex[:].copy(),tildey[:].copy()
        ## >>>> renormalization, mu_l and mu_r are unchanged
        xref,yref = np.reshape(xAm[:,irank].copy(),(NE*2,1)),np.reshape(yAm[:,irank].copy(),(NE*2,1))
        tildex,tildey = np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
        x0[:,irank]=np.squeeze(np.sqrt(np.squeeze(tildey.T@xref)/np.squeeze(yref.T@tildex))*tildex)
        y0[:,irank]=np.squeeze(np.sqrt(np.squeeze(tildex.T@yref)/np.squeeze(xref.T@tildey))*tildey)
    for idxeta,eta in enumerate(etaseries):
        etaset=eta*np.ones(3)#np.zeros(3)#eta*np.ones(3)
        for icoeff in range(3):
            etaset[icoeff]*=coeffeta[icoeff]
        # etaset[0]*=0.1
        Xinit = Xsym.copy()
        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ## EI IE##
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()

        X=Xinit.copy()
        # incase heterogeneous
        X[:NE,:NE]*=(xee)
        X[:NE,NE:]*=(xei)
        X[NE:,:NE]*=(xie)
        X[NE:,NE:]*=(xii)

        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T

        # properties
        ''' method 1 '''
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        # ''' method 2 '''
        # eigvJ,eiglvecJ,eigrvecJ=scpla.eig(J,left=True,right=True)
        # meig = np.squeeze(eigrvecJ[:,:].copy())
        # neig = np.squeeze(eiglvecJ[:,:].copy()) # inverse

        # print('test normal 4:',np.sum(neig[:,0]*meig[:,0]))
        for i in range(nrank):
            neig[:,i]=neig[:,i]*eigvJ[i]
        for j in range(nrank,N):
            neig[:,j]=neig[:,j]*eigvJ[j]
        '''
        Sort Eigenvalue in ascending 
        '''
        eigenvalue_Rsort = np.squeeze(-np.abs(np.real(eigvJ.copy())))
        idxsort=np.argsort(eigenvalue_Rsort)
        eigenvalue_sort = np.squeeze(eigvJ.copy())
        # eigvalseries[idxeta,iktrial,:] = eigenvalue_sort[idxsort]
        eigvJ= eigvJ[idxsort]  #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        # print('perturbed eigenvalue:',eigvJ[:nrank])
        meig = meig[:,idxsort] #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        neig = neig[:,idxsort] #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        ## >>> for eigendecomposition 
        Leig, Reig = np.zeros((N,N),dtype=complex),np.zeros((N,N),dtype=complex)
        Reig = meig.copy()
        Leig = neig.copy()
        normval=np.sum(Reig*Leig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=Leig.copy()/normval.copy()  # note that here we change from neig --> Leig, otherwise, if we use copy, there'll be some sort of mistakes
        # print('test normal 5:',np.sum(Leig[:,0]*Reig[:,0]))
        '''## for reference '''
        xnorm0,ynorm0=np.zeros((NE*2,nrank*nrec),dtype=complex),np.zeros((NE*2,nrank*nrec),dtype=complex)
        for irank in range(nrank*2):
            xnorm0[:,irank],ynorm0[:,irank]=Reig[:,irank].copy(),Leig[:,irank].copy()
            ''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
            tildex,tildey=xnorm0[:,irank].copy(),ynorm0[:,irank].copy()

            ## make sure the E in y is positive, if negative then all x and y should change sign
            # if (np.mean(np.real(tildey[:NE]))*np.real(yAm[1,0]))<0:
            #     tildex*=(-1)
            #     tildey*=(-1)
            # xnorm0[:,irank],ynorm0[:,irank]=tildex[:].copy(),tildey[:].copy()

            xref,yref = np.reshape(xAm[:,0].copy(),(NE*2,1)),np.reshape(yAm[:,0].copy(),(NE*2,1))
            tildex,tildey = np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
            xnorm0[:,irank]=np.squeeze(np.sqrt(np.squeeze(tildey.T@xref)/np.squeeze(yref.T@tildex))*tildex)
            ynorm0[:,irank]=np.squeeze(np.sqrt(np.squeeze(tildex.T@yref)/np.squeeze(xref.T@tildey))*tildey)
            if (np.mean(np.real(ynorm0[:NE,irank]))*np.real(yAm[0,0]))<0:
                ynorm0[:,irank]*=(-1)
                xnorm0[:,irank]*=(-1)


        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        Reigvecseries[idxeta,iktrial,:,:],Leigvecseries[idxeta,iktrial,:,:]=xnorm0[:,:].copy(),ynorm0[:,:].copy()
        print('norm sum last:',np.sum(Reigvecseries[idxeta,iktrial,:,:]*Leigvecseries[idxeta,iktrial,:,:]))
        if (idxeta==0):
            ''' record the random derivative '''
            print('recording!!!')
            ## log: there is a bug, X is the total random xconn, including symmetry
            gYbasexee[iktrial,:,:]=Xsym.copy()


normitems (0.0028206792251134513+0.00038650006699888j) ; eigenvalue: (0.3750000000000011+0.1198957880828219j) (0.3749999999999961+0.11989578808281057j)
normitems (-0.03359788359348363+0.004603708264407712j) ; eigenvalue: (0.3750000000000011-0.1198957880828219j) (0.3749999999999961-0.11989578808281057j)
eigenvalues of unperturbed matrix: [0.375+0.11989579j 0.375-0.11989579j]
test normal unperturbed: (1.0000000000000004+0j) (1.0000000000000004+0j)
test normal: (0.9999999999998019-3.816946758661288e-13j)
norm sum last: (4.0000000000000036-5.120957911192159e-16j)
recording!!!
norm sum last: (4.000000000000005-4.707334782388939e-16j)
norm sum last: (4.000000000000001-3.0292258760486853e-28j)
norm sum last: (3.999999999999991+2.8272774843121063e-27j)
norm sum last: (3.999999999999997-8.326672684688674e-17j)
norm sum last: (4.000000000000003+2.4318654728838505e-16j)
norm sum last: (4.000000000000001+5.427363027920561e-28j)
norm sum last: (4.000000000000001-1.8214596497756474e-17j)
norm sum la

In [328]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

# ''' >>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>> '''

# np.savez(now+'_perturbationSubcircuitChangeIeta_data',x0series=x0series,y0series=y0series,xAm=xAm,yAm=yAm,Jparameters=Jparameters,Randgain=Randgain,gaverage=gaverage,eta=eta,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,Beigvseries=np.squeeze(Beigvseries[:,:,:2]),meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],gYbasexee=gYbasexee,SparseMat=SparseMat,coeffetaEsub=coeffetaEsub,coeffetaTotal=coeffetaTotal,signetaEsub=signetaEsub,signetaTotal=signetaTotal,ppercentEsub=ppercentEsub,coeffetaIseries=coeffetaIseries)
nvec = nrank*2
np.savez(now+'_perturbationSecondVector_KM_data',xAm=xAm,yAm=yAm,eigvAm = eigvAm[0],Jparameters=Jparameters,Randgain=Randgain,gaverage=gaverage,eta=eta,Reigvecseries=Reigvecseries,Leigvecseries=Leigvecseries,Beigvseries=np.squeeze(Beigvseries[:,:,:nvec]),meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],gYbasexee=gYbasexee,coeffeta=coeffeta,signeta=signeta,etaseries=etaseries)

In [297]:
# '''
# with symmetry, 
# eigvJ0
# '''
# idxgavg,idxtrial=9,8 # 6,3,0
# figtspt,axtspt=plt.subplots(figsize=(6,3))
# cm='bgc'
# idxc=0
# axtspt.scatter(np.real(eigvJ0),np.imag(eigvJ0),s=5,c=cm[idxc],alpha=0.25) # >>>>>>>>>>>>>>
# for irank in range(nrank):
#     axtspt.scatter(np.real(eigvJ0[irank]),np.imag(eigvJ0[irank])-0.2*idxc,s=20,c=cm[1],marker='^',alpha=0.5) # >>>>>>>>>>>>

# axtspt.set_aspect('equal')
# axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
# axtspt.spines['right'].set_color('none')
# axtspt.spines['top'].set_color('none')
# axtspt.xaxis.set_ticks_position('bottom')
# axtspt.spines['bottom'].set_position(('data', 0))

# idxgavg,idxtrial=9,8 # 6,3,0
# figtspt,axtspt=plt.subplots(figsize=(6,3))
# cm='bgc'
# idxc=0
# axtspt.scatter(np.real(Beigvseries[idxtrial,1:]),np.imag(Beigvseries[idxtrial,1:]),s=5,c=cm[idxc],alpha=0.25) # >>>>>>>>>>>>>>
# axtspt.scatter(np.real(Beigvseries[:,0]),np.imag(Beigvseries[:,0])-0.2*idxc,s=20,c=cm[idxc+1],marker='^',alpha=0.5) # >>>>>>>>>>>>
# axtspt.scatter(np.real(Beigvseries[:,1]),np.imag(Beigvseries[:,1])-0.2*idxc,s=20,c=cm[idxc+2],marker='^',alpha=0.5) # >>>>>>>>>>>>
# axtspt.set_aspect('equal')

# axtspt.spines['right'].set_color('none')
# axtspt.spines['top'].set_color('none')
# axtspt.xaxis.set_ticks_position('bottom')
# axtspt.spines['bottom'].set_position(('data', 0))



'''
with symmetry, subcircuit
eigvJ0
'''
cm='rbg'
idxc=0
idxgavg,idxtrial=0,17 # 6,3,0
figtspt,axtspt=plt.subplots(figsize=(4,4))
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,:]),np.imag(Beigvseries[idxgavg,idxtrial,:]),s=2,c=cm[idxc+1],alpha=0.25) # >>>>>>>>>>>>>>
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,0]),np.imag(Beigvseries[idxgavg,idxtrial,0])-0.2*idxc,s=20,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1]),np.imag(Beigvseries[idxgavg,idxtrial,1])-0.2*idxc,s=20,c='darkblue',marker='^',alpha=0.5) # >>>>>>>>>>>>

axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
axtspt.xaxis.set_ticks_position('bottom')
axtspt.spines['bottom'].set_position(('data', 0))
xticks = np.linspace(-1.5,1.5,2)
xlims = [-1.5,1.5]
yticks = np.linspace(-1.0,1.0,2)
ylims = [-1.0,1.0]
axtspt.set_xlim(xlims)
axtspt.set_ylim(ylims)
axtspt.set_xticks(xticks)
axtspt.set_yticks(yticks)
axtspt.set_aspect('equal')

'''
with symmetry, subcircuit
eigvJ0
'''
cm='rbg'
idxc=0
print(ntrial)
idxgavg,idxtrial=9,8 # 6,3,0
for idxgavg in range(neta):
    figtspt,axtspt=plt.subplots(figsize=(4,4))
    for idxtrial in range(ntrial):
        axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,:]),np.imag(Beigvseries[idxgavg,idxtrial,:]),s=1,c=cm[idxc+1],alpha=0.25) # >>>>>>>>>>>>>>
    axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0])-0.2*idxc,s=20,c='red',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.scatter(np.real(Beigvseries[idxgavg,:,1]),np.imag(Beigvseries[idxgavg,:,1])-0.2*idxc,s=20,c='darkblue',marker='^',alpha=0.5) # >>>>>>>>>>>>
    axtspt.set_aspect('equal')
    axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
    axtspt.spines['right'].set_color('none')
    axtspt.spines['top'].set_color('none')
    axtspt.xaxis.set_ticks_position('bottom')
    axtspt.spines['bottom'].set_position(('data', 0))
    xticks = np.linspace(-1.5,1.5,2)
    xlims = [-1.5,1.5]
    yticks = np.linspace(-1.0,1.0,2)
    ylims = [-1.0,1.0]
    axtspt.set_xlim(xlims)
    axtspt.set_ylim(ylims)
    axtspt.set_xticks(xticks)
    axtspt.set_yticks(yticks)
    axtspt.set_aspect('equal')

/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the 

In [226]:
'''
determinant lemma, odd terms 
even order terms should be maitained
'''
xticks = np.linspace(-2.91,-2.5,2)
xlims = [-2.91,-2.5]
yticks = np.linspace(-2.91,-2.5,2)
ylims = [-2.91,-2.5]

coeff=coeffeta.copy()
Dcoeff = np.zeros_like(etaseries)
# nAm,mAm rank two 
nAm,mAm = yAm.copy(),xAm.copy()
nAm[:NE,0],nAm[NE:,0]=2*JE/np.sqrt(NE*2),-2*JI/np.sqrt(NE*2)
nAm[:NE,1],nAm[NE:,1]=2*a/np.sqrt(NE*2),2*b/np.sqrt(NE*2)
mAm[:NE,0],mAm[NE:,0]=1/np.sqrt(NE*2),1/np.sqrt(NE*2)
mAm[:NE,1],mAm[NE:,1]=1/np.sqrt(NE*2),-1/np.sqrt(NE*2)
lambda_theo_simple = np.zeros((neta,3),dtype=complex)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:,:nrank]),axis=1)
cutoff = 2

for idxeta,eta in enumerate(etaseries):
    etaE,etaB,etaI=eta*coeffeta[0],eta*coeffeta[1],eta*coeffeta[2]
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    C=comb(2,1)
    bXXT=np.eye(NE*2)
    bXXT[:NE,:NE]*=(etaE*signeta[0]+etaB*signeta[1])/4
    bXXT[NE:,NE:]*=(etaI*signeta[2]+etaB*signeta[1])/4
    Mlambda0,Mlambda2 = np.eye(NE*2),bXXT.copy()*gaverage**2
    Qlambda0,Qlambda2 = nAm.T@Mlambda0@mAm,C*nAm.T@Mlambda2@mAm # /lambda^2

    print('second moment:',Qlambda2)
    print('predict second moment:',(JE*(etaE*signeta[0]+etaB*signeta[1])/4.0-JI*(etaI*signeta[2]+etaB*signeta[1])/4.0)*gaverage**2*2)
    bcoeff = -(JE-JI)
    dcoeff = -((JE*(etaE*signeta[0]+etaB*signeta[1])/4.0-JI*(etaI*signeta[2]+etaB*signeta[1])/4.0)*gaverage**2*2)
    roots = Cubic([1,bcoeff,0,dcoeff])
    print('predict root:',roots)
    print('numerical:',lambda_num[idxeta,:])
    lambda_theo_simple[idxeta,:] = roots
    Dcoeff[idxeta]=dcoeff

# cm = 'bgc'
# fig,ax=plt.subplots(figsize=(4,4)) 
# for idx in range(3):
#     if idx<2:
#         for iktrial in range(ntrial):
#             ax.plot(etaseries,np.real(Beigvseries[:,iktrial,idx]),'^',color=cm[idx],markersize=2,alpha=0.5)
#     ax.plot(etaseries,np.real(lambda_theo_simple[:,idx]),color=cm[idx])
# ax.set_aspect('equal')


# fig,ax=plt.subplots(figsize=(4,4)) 
# for idx in range(3):
#     if idx<2:
#         for iktrial in range(ntrial):
#             ax.plot(etaseries,np.imag(Beigvseries[:,iktrial,idx]),'^',color=cm[idx],markersize=2,alpha=0.5)
#     ax.plot(etaseries,np.imag(lambda_theo_simple[:,idx]),color=cm[idx])
# ax.set_aspect('equal')

cm = 'bgc'
fig,ax=plt.subplots(figsize=(4,4)) 
for idx in range(3):
    if idx==0:
        for iktrial in range(ntrial):
            ax.plot(Dcoeff,np.real(Beigvseries[:,iktrial,idx]),'^',color='k',markersize=5,alpha=0.5)
    if idx==1:
        for iktrial in range(ntrial):
            ax.plot(Dcoeff,np.real(Beigvseries[:,iktrial,idx]),'o',color='k',markersize=5,alpha=0.5)
    if idx ==0:
        ax.plot(Dcoeff,np.real(lambda_theo_simple[:,idx]),color=cm[1],linewidth = 2.0)
    else:
        ax.plot(Dcoeff,np.real(lambda_theo_simple[:,idx]),color=cm[1],linewidth = 1.0,linestyle = '--')
# ax.set_aspect('equal')


fig,ax=plt.subplots(figsize=(4,4)) 
for idx in range(3):
    if idx==0:
        for iktrial in range(ntrial):
            ax.plot(Dcoeff,np.imag(Beigvseries[:,iktrial,idx]),'^',color='k',markersize=5,alpha=0.5)
    if idx==1:
        for iktrial in range(ntrial):
            ax.plot(Dcoeff,np.imag(Beigvseries[:,iktrial,idx]),'o',color='k',markersize=5,alpha=0.5)
    if idx ==0:
        ax.plot(Dcoeff,np.imag(lambda_theo_simple[:,idx]),color=cm[1],linewidth = 2.0)
    else:
        ax.plot(Dcoeff,np.imag(lambda_theo_simple[:,idx]),color=cm[1],linewidth = 1.0,linestyle = '--')
# ax.set_aspect('equal')


second moment: [[0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
predict second moment: 0.0
predict root: [0.65, (-0+0j), (-0-0j)]
numerical: [ 0.79897212+0.j         -0.03586396+0.00690158j]
second moment: [[0.01667361+0.j 0.01667361+0.j]
 [0.00170139+0.j 0.00170139+0.j]]
predict second moment: 0.01667361111111111
predict root: [0.685484, (-0.017742+0.154949j), (-0.017742-0.154949j)]
numerical: [ 0.82720487+0.j         -0.07565829+0.00272317j]
second moment: [[0.03334722+0.j 0.03334722+0.j]
 [0.00340278+0.j 0.00340278+0.j]]
predict second moment: 0.03334722222222222
predict root: [0.715195, (-0.032597+0.213458j), (-0.032597-0.213458j)]
numerical: [ 0.85323541+0.j         -0.08051442-0.00094775j]
second moment: [[0.05002083+0.j 0.05002083+0.j]
 [0.00510417+0.j 0.00510417+0.j]]
predict second moment: 0.05002083333333333
predict root: [0.74108, (-0.04554+0.25578j), (-0.04554-0.25578j)]
numerical: [ 0.87755815+0.j         -0.04370922+0.00397042j]
second moment: [[0.06669444+0.j 0.06669444+0.j]
 [0.00680

In [190]:
nlambda_axis_real = 51
nlambda_axis_imag = 31

lambda_axis_real = np.linspace(0.0,1.0,nlambda_axis_real)
hreal = lambda_axis_real[1]-lambda_axis_real[0]
lambda_axis_imag = np.linspace(-0.5,0.5,nlambda_axis_imag)
himag = lambda_axis_imag[1]-lambda_axis_imag[0]
Qlambda_flambda= np.zeros((nlambda_axis_real,nlambda_axis_imag,2,2),dtype=complex)
func_lambda_rhs = np.zeros((nlambda_axis_real,nlambda_axis_imag),dtype=complex)
lambda_surface = np.zeros((nlambda_axis_real,nlambda_axis_imag),dtype=complex)
epsilon=1e-3
idxs = 0
for idxeta,eta in enumerate(etaseries[idxs::10]):
    etaE,etaB,etaI=eta*coeffeta[0],eta*coeffeta[1],eta*coeffeta[2]
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    C=comb(2,1)
    bXXT=np.eye(NE*2)
    bXXT[:NE,:NE]*=(etaE*signeta[0]+etaB*signeta[1])/4
    bXXT[NE:,NE:]*=(etaI*signeta[2]+etaB*signeta[1])/4
    Mlambda0,Mlambda2 = np.eye(NE*2),bXXT.copy()*gaverage**2
    for ilambdaR in range(nlambda_axis_real):
        for ilambdaI in range(nlambda_axis_imag):
            lambda_axis = np.complex(lambda_axis_real[ilambdaR],lambda_axis_imag[ilambdaI])
            lambda_axis2 = lambda_axis**2#lambda_axis_real[ilambdaR]**2+lambda_axis_imag[ilambdaI]**2
            Qlambda_flambda[ilambdaR,ilambdaI,:,:] = nAm.T@(Mlambda0+C*Mlambda2/lambda_axis2)@mAm
            treq = (Qlambda_flambda[ilambdaR,ilambdaI,0,0]+Qlambda_flambda[ilambdaR,ilambdaI,1,1])
            deteq = (Qlambda_flambda[ilambdaR,ilambdaI,0,0]*Qlambda_flambda[ilambdaR,ilambdaI,1,1])-(Qlambda_flambda[ilambdaR,ilambdaI,0,1]*Qlambda_flambda[ilambdaR,ilambdaI,1,0])
            func_lambda_rhs[ilambdaR,ilambdaI]=lambda_axis*treq-deteq
            lambda_surface[ilambdaR,ilambdaI] = lambda_axis2
    errorR = (np.real(func_lambda_rhs)-np.real(lambda_surface))**2
    idxR,idyR = np.where(np.sqrt(errorR)<epsilon)
    errorI = (np.imag(func_lambda_rhs)-np.imag(lambda_surface))**2
    idxI,idyI = np.where(np.sqrt(errorI)<epsilon)
    sqrterrorRI = np.sqrt(errorR+errorI)
    idxRI,idyRI = np.where(sqrterrorRI<epsilon)
    fig,ax=plt.subplots(figsize=(5,5))
    ax.imshow(-np.log(np.sqrt(errorR+errorI)),vmax=6.0,vmin=0.0)
    ax.plot((np.imag(Beigvseries[idxeta+idxs,:,0])-lambda_axis_imag[0])/himag,(np.real(Beigvseries[idxeta+idxs,:,0])-lambda_axis_real[0])/hreal,'^',color='red',markersize=3)
    ax.plot((np.imag(Beigvseries[idxeta+idxs,:,1])-lambda_axis_imag[0])/himag,(np.abs(np.real(Beigvseries[idxeta+idxs,:,1]))-lambda_axis_real[0])/hreal,'^',color='red',markersize=3)
    # ax.scatter(idyI,idxI,s=5,c='darkred')
    # ax.scatter(idyR,idxR,s=5,c='darkblue')
    # fig,ax=plt.subplots(figsize=(5,5))
    # ax.imshow(errorI)
    # ax.scatter(idxI,idyI,s=5,c='darkred')
    # fig,ax=plt.subplots(figsize=(5,5))  
    # ax.plot(lambda_axis,np.sqrt(func_lambda_rhs),'b',marker='o',markersize=3)
    # ax.plot(lambda_axis,lambda_axis,'c')

<ipython-input-190-52932f195043>:27: RuntimeWarning: invalid value encountered in true_divide
  Qlambda_flambda[ilambdaR,ilambdaI,:,:] = nAm.T@(Mlambda0+C*Mlambda2/lambda_axis2)@mAm
<ipython-input-190-52932f195043>:33: RuntimeWarning: invalid value encountered in less
  idxR,idyR = np.where(np.sqrt(errorR)<epsilon)
<ipython-input-190-52932f195043>:35: RuntimeWarning: invalid value encountered in less
  idxI,idyI = np.where(np.sqrt(errorI)<epsilon)
<ipython-input-190-52932f195043>:37: RuntimeWarning: invalid value encountered in less
  idxRI,idyRI = np.where(sqrterrorRI<epsilon)


In [28]:
idxs = 5
fig,ax=plt.subplots(figsize=(5,5))
ax.imshow(-np.log(np.sqrt(errorR+errorI)),vmax=6.0,vmin=0.0)
ax.plot((np.imag(Beigvseries[idxeta+idxs,:,0])-lambda_axis_imag[0])/himag,(np.real(Beigvseries[idxeta+idxs,:,0])-lambda_axis_real[0])/hreal,'^',color='red',markersize=5)
ax.plot((np.imag(Beigvseries[idxeta+idxs,:,1])-lambda_axis_imag[0])/himag,(np.abs(np.real(Beigvseries[idxeta+idxs,:,1]))-lambda_axis_real[0])/hreal,'^',color='red',markersize=5)

In [104]:
# OVERLAP MATRICES)
Qoverlap = nAm.T@(C*Mlambda2/np.real(eigvJ0[0])**2)@mAm
eigvQ,eigvecQ = la.eig(Qoverlap)
print(eigvQ)
print(Beigvseries[8,0]-eigvJ0[0])

[0.1506926 +0.j 0.00525048+0.j]
(0.13213848270422202+0j)


In [331]:
# strfile = '2021-04-24-17_45_45_perturbationSecondVectorChangeIeta_data'
# strfile = '2021-04-24-17_04_23_perturbationSecondVectorChangeIeta_data'

# data = np.load(strfile+'.npz')
# etaseries=data['etaseries'],
# neta=len(etaseries)
# nrank=2
# xAm,yAm = data['xAm'],data['yAm']
# gYbasexee=data['gYbasexee']
# Jparameters,Randgain=data['Jparameters'],data['Randgain']
# NE = int(Jparameters[4])
# xee,xei,xie,xii=Randgain[0],Randgain[1],Randgain[2],Randgain[3]
# # right and left eigenvectors
# Reigvecseries,Leigvecseries,Beigvseries=data['Reigvecseries'],data['Leigvecseries'],data['Beigvseries']
# ntrial=np.shape(Reigvecseries)[1]
# coeffeta,signeta = data['coeffeta'],data['signeta']
# eigvAm = data['eigvAm']

'''
loading patterns
for ensemble-averaging results
>>>>>>>>>>>>>>>>> with eigenvalues >>>>>>>>>>>>>>>
'''
# ## strfile = '2021-04-24-13_17_52_perturbationSecondVectorChangeIeta_data'
# xticks = np.linspace(-0.3,0.4,2)
# xlims = [-0.3,0.4]
# yticks = np.linspace(-0.2,0.3,2)
# ylims = [-0.2,0.3]

## strfile = '2021-04-24-15_07_44_perturbationSecondVectorChangeIeta_data'
xticks = np.linspace(-1.0,1.0,2)
xlims = [-1.0,1.0]
yticks = np.linspace(-0.3,0.4,2)
ylims = [-0.3,0.4]

# xticks = np.linspace(-0.25,0.3,2)
# xlims = [-0.25,0.3]
# yticks = np.linspace(-0.2,0.3,2)
# ylims = [-0.2,0.3]

theta = np.linspace(0, 2 * np.pi, 200)
''' how eigenvalue outlier changes with g_bar '''
cm='kbgmr'
idm= 1
idxtrial=12#16#
idxeta=6#1# 
alphaval=0.10
edgv=cm[0]#[idm]#

# nrank=1
ntrial = np.shape(Reigvecseries)[1]
NE=int(NE)

'''loading vector changing'''
meanmE,meanmI,meannE,meannI=np.zeros(nrank*nrec),np.zeros(nrank*nrec),np.zeros(nrank*nrec),np.zeros(nrank*nrec)


# mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxeta,:,:NE,idm]),np.squeeze(Reigvecseries[idxeta,:,NE:,idm]),np.squeeze(Leigvecseries[idxeta,:,:NE,idm]),np.squeeze(Leigvecseries[idxeta,:,NE:,idm])

mEvec,mIvec,nEvec,nIvec=np.squeeze(Reigvecseries[idxeta,idxtrial,:NE,:]),np.squeeze(Reigvecseries[idxeta,idxtrial,NE:,:]),np.squeeze(Leigvecseries[idxeta,idxtrial,:NE,:]),np.squeeze(Leigvecseries[idxeta,idxtrial,NE:,:])
scale_std=3.0
for irank in range(nrank*nrec):
    meanmEtotal,stdmEtotal = np.mean(mEvec[:,irank]),np.std(mEvec[:,irank])
    varmE = mEvec - meanmEtotal
    idxwhere = np.where(np.abs(varmE)>scale_std*stdmEtotal)
    mEvec[idxwhere,irank]=meanmEtotal
    meanmE[irank]=np.mean(mEvec[:,irank])

    # puring I
    meanmItotal,stdmItotal = np.mean(mIvec[:,irank]),np.std(mIvec[:,irank])
    varmI = mIvec - meanmItotal
    idxwhere = np.where(np.abs(varmI)>scale_std*stdmItotal)
    mIvec[idxwhere,irank]=meanmItotal
    meanmI[irank]=np.mean(mIvec[:,irank])
    
    # n vector
    # puring E
    meannEtotal,stdnEtotal = np.mean(nEvec[:,irank]),np.std(nEvec[:,irank])
    varnE = nEvec - meannEtotal
    idxwhere = np.where(np.abs(varnE)>scale_std*stdnEtotal)
    nEvec[idxwhere,irank]=meannEtotal
    meannE[irank]=np.mean(nEvec[:,irank])

    # puring I
    meannItotal,stdnItotal = np.mean(nIvec[:,irank]),np.std(nIvec[:,irank])
    varnI = nIvec - meannItotal
    idxwhere = np.where(np.abs(varnI)>scale_std*stdnItotal)
    nIvec[idxwhere,irank]=meannItotal
    meannI[irank]=np.mean(nIvec[:,irank])

''' calculate directions '''
# noiseE,noiseI = np.zeros((NE*ntrial,2,nrank)),np.zeros((NE*ntrial,2,nrank))
noiseE,noiseI = np.zeros((NE,2,nrank*nrec)),np.zeros((NE,2,nrank*nrec))
dirvecE,dirvecI=np.zeros((2,2,nrank*nrec)),np.zeros((2,2,nrank*nrec))
for irank in range(nrank*nrec):
    # E 0 M 1 N
    noiseE[:,0,irank],noiseE[:,1,irank]= nEvec[:,irank]-meannE[irank],mEvec[:,irank]-meanmE[irank]
    noiseI[:,0,irank],noiseI[:,1,irank]= nIvec[:,irank]-meannI[irank],mIvec[:,irank]-meanmI[irank]
    # m1n1
    covdirE,covdirI=np.squeeze(noiseE[:,:,irank]).T@np.squeeze(noiseE[:,:,irank]),np.squeeze(noiseI[:,:,irank]).T@np.squeeze(noiseI[:,:,irank])
    _,dirvecE[:,:,irank]=la.eig(covdirE)
    _,dirvecI[:,:,irank]=la.eig(covdirI)  


for irank in range(nrank*nrec):
    fig,ax=plt.subplots(figsize=(5,3))  
    ax.scatter(nIvec[:,irank],mIvec[:,irank],s=5.0,c='b',alpha=alphaval)#cm[1],alpha=alphaval)
    ax.scatter(nEvec[:,irank],mEvec[:,irank],s=5.0,c='r',alpha=alphaval)#cm[0],alpha=alphaval)

    ax.plot([meannE[irank],meannE[irank]+dirvecE[0,0,irank]],[meanmE[irank],meanmE[irank]+dirvecE[1,0,irank]],color='darkred',linestyle='--',linewidth=2.0)
    ax.plot([meannE[irank],meannE[irank]+dirvecE[0,1,irank]],[meanmE[irank],meanmE[irank]+dirvecE[1,1,irank]],color='darkred',linestyle='--',linewidth=2.0)

    ax.plot([meannI[irank],meannI[irank]+dirvecI[0,0,irank]],[meanmI[irank],meanmI[irank]+dirvecI[1,0,irank]],color='darkblue',linestyle='--',linewidth=2.0)
    ax.plot([meannI[irank],meannI[irank]+dirvecI[0,1,irank]],[meanmI[irank],meanmI[irank]+dirvecI[1,1,irank]],color='darkblue',linestyle='--',linewidth=2.0)

    confidence_ellipse(np.real(nEvec[:,irank]),np.real(mEvec[:,irank]),ax,edgecolor='red')#edgv)
    confidence_ellipse(np.real(nIvec[:,irank]),np.real(mIvec[:,irank]),ax,edgecolor='blue')#edgv)
    ax.axvline(c='grey', lw=1)
    ax.axhline(c='grey', lw=1)
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)
    ax.set_aspect('equal')
print(Beigvseries[idxeta,idxtrial,idm-1])
print(Beigvseries[idxeta,idxtrial,idm])
print(Beigvseries[idxeta,idxtrial,idm+1])

print('eta:',etaseries[idxeta])

(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(2, 2)
(0.7762420853106605+0j)
(-0.514766895090601+0.009865139341811817j)
(-0.514766895090601-0.009865139341811817j)
eta: 0.6666666666666666
<ipython-input-331-11b727ea692d>:67: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmE[irank]=np.mean(mEvec[:,irank])
<ipython-input-331-11b727ea692d>:74: ComplexWarning: Casting complex values to real discards the imaginary part
  meanmI[irank]=np.mean(mIvec[:,irank])
<ipython-input-331-11b727ea692d>:82: ComplexWarning: Casting complex values to real discards the imaginary part
  meannE[irank]=np.mean(nEvec[:,irank])
<ipython-input-331-11b727ea692d>:89: ComplexWarning: Casting complex values to real discards the imaginary part
  meannI[irank]=np.mean(nIvec[:,irank])
<ipython-input-331-11b727ea692d>:97: ComplexWarning: Casting complex values to real discards the imaginary part
  noiseE[:,0,irank],noiseE[:,1,irank]= nEvec[:,irank]-meannE[irank],mEvec[:,irank]-meanmE[

In [313]:
## overlap between left and right eigenvectors ( normalized )
DeltaL_num,DeltaR_num=np.zeros((neta,ntrial,NE*2),dtype=complex),np.zeros((neta,ntrial,NE*2),dtype=complex) 
DeltaLR_num = np.zeros((neta,ntrial,3),dtype=complex)
Delta_lambda = np.zeros((neta,ntrial,1),dtype=complex)

changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,idxtrial,:,0].copy()),(NE*2,1))#-l0#np.mean(np.squeeze(Leigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Leigvecseries[0,itrial,:]),(NE*2,1))
changeL_num[:NE,0]-=np.mean(changeL_num[:NE,0])
changeL_num[NE:,0]-=np.mean(changeL_num[NE:,0])
changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,idxtrial,:,0].copy()),(NE*2,1))#-r0#np.mean(np.squeeze(Reigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Reigvecseries[0,itrial,:]),(NE*2,1))
changeR_num[:NE,0]-=np.mean(changeR_num[:NE,0])
changeR_num[NE:,0]-=np.mean(changeR_num[NE:,0])

DeltaLR_num[idxeta,idxtrial,2]=np.squeeze(changeL_num.T@changeR_num)
DeltaLR_num[idxeta,idxtrial,0]=np.sum(changeL_num[:NE,0]*changeR_num[:NE,0])
DeltaLR_num[idxeta,idxtrial,1]=np.sum(changeL_num[NE:,0]*changeR_num[NE:,0])

changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,idxtrial,:,1].copy()),(NE*2,1))#-l0#np.mean(np.squeeze(Leigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Leigvecseries[0,itrial,:]),(NE*2,1))
changeL_num[:NE,0]-=np.mean(changeL_num[:NE,0])
changeL_num[NE:,0]-=np.mean(changeL_num[NE:,0])
changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,idxtrial,:,1].copy()),(NE*2,1))#-r0#np.mean(np.squeeze(Reigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Reigvecseries[0,itrial,:]),(NE*2,1))
changeR_num[:NE,0]-=np.mean(changeR_num[:NE,0])
changeR_num[NE:,0]-=np.mean(changeR_num[NE:,0])

DeltaLR_num[idxeta,idxtrial,2]+=np.squeeze(changeL_num.T@changeR_num)
DeltaLR_num[idxeta,idxtrial,0]+=np.sum(changeL_num[:NE,0]*changeR_num[:NE,0])
DeltaLR_num[idxeta,idxtrial,1]+=np.sum(changeL_num[NE:,0]*changeR_num[NE:,0])

etalambda =Beigvseries[idxeta,idxtrial,0]
Delta_lambda[idxeta,idxtrial,0]= (Beigvseries[idxeta,idxtrial,0]-1*np.real(eigvAm[0]))/np.real(1*eigvAm[0])#*Beigvseries[idxeta,idxtrial,0]**2/np.real(2*eigvAm[0])**2

print('overlap:',DeltaLR_num[idxeta,idxtrial,2],Delta_lambda[idxeta,idxtrial,0])





overlap: (1.3206746052569875+4.879200093406301e-15j) (1.2349263260205057+0j)


In [267]:
print(meanmE[0]*meannE[0]+meanmI[0]*meannI[0],JE-JI)

0.0009222712943497123 0.6500000000000001


In [39]:
print(np.squeeze(Leigvecseries[idxeta,:,:NE,idm]))

[[        nan+0.j         nan+0.j         nan+0.j ...         nan+0.j
          nan+0.j         nan+0.j]
 [-0.2469412 +0.j -0.13032267+0.j -0.19693558+0.j ... -0.10426574+0.j
  -0.15281144+0.j -0.24465074+0.j]
 [-0.19121184+0.j -0.06772397+0.j -0.14799745+0.j ... -0.07085458+0.j
  -0.2910095 +0.j -0.16766189+0.j]
 ...
 [        nan+0.j         nan+0.j         nan+0.j ...         nan+0.j
          nan+0.j         nan+0.j]
 [-0.20611994+0.j -0.25530624+0.j -0.15696056+0.j ... -0.39754851+0.j
  -0.10606828+0.j -0.59458766+0.j]
 [-0.21324913+0.j -0.19041884+0.j -0.26840385+0.j ... -0.49782798+0.j
  -0.44691261+0.j -0.07705099+0.j]]
